In [1]:
from utils.ReviewRBInit import ReviewRBInit
from utils.ReviewRBLoop import ReviewRBLoop
from utils.ReviewMLInit import ReviewMLInit
from utils.ReviewMLLoop import ReviewMLLoop
from gui.Workflow import Workflow
from sqlalchemy_dao import Dao
from db.ORMs import Document
from utils.IntroStep import IntroStep
from gui.PreviousNextWidgets import PreviousNextHTML
import sqlalchemy_dao
import os
from  conf.ConfigReader import ConfigReader
from utils.KeywordsUMLSExtender import KeywordsUMLSExtender
from utils.KeywordsUMLSExtenderSetup import KeywordsUMLSExtenderSetup
from utils.TreeSet import TreeSet
from utils.IntroStep import IntroStep
import logging
logging.getLogger().setLevel(logging.DEBUG)
ConfigReader('../conf/smartanno_conf.json')

from models.GloveModel import GloveModel
from  conf.ConfigReader import ConfigReader
from threading import Thread


wf=Workflow()
wf.filters={'TypeA':TreeSet(['MI','burned']),'TypeB':TreeSet(['filaria'])}
wf.append(KeywordsUMLSExtenderSetup(name='umls_extender_setup'))
wf.append(KeywordsUMLSExtender(name='umls_extender', sources=ConfigReader.getValue("umls/sources"),
                  filter_by_length=ConfigReader.getValue("umls/filter_by_length"),
                  filter_by_contains=ConfigReader.getValue("umls/filter_by_contains"),
                  max_query=ConfigReader.getValue("umls/max_query")))
wf.append(IntroStep('<h2>Welcome to SmartAnno!</h2><h4>First, let&apos;s import txt data from a directory. </h4>',
                       name='intro'))
wf.start()

VBox(children=(Label(value='', layout=Layout(height='5px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HTML(value='<h2>Welcome to SmartAnno!</h2><h4>First, let&apos;s import txt data from a directory. </h4>'), Label(value='', layout=Layout(height='5px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HBox(children=(Button(description='StartOver', layout=Layout(left='100px', width='150px'), style=ButtonStyle()), Button(description='ContinueReviewing', layout=Layout(left='100px', width='150px'), style=ButtonStyle())), layout=Layout(left='10%', width='80%'))), layout=Layout(display='flex'))

In [2]:
wf.filters

{'TypeA': ['MI', 'burned'], 'TypeB': ['filaria']}